In [2]:
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role

# 从环境变量中，获取 DASHSCOPE_API_KEY
api_key = os.environ.get('DASHSCOPE_API_KEY')
dashscope.api_key = api_key

# 编写你的天气函数
# 为了演示流程，这里指定了天气的温度，实际上可以调用 高德接口获取实时天气。
# 这里可以先用每个城市的固定天气进行返回，查看大模型的调用情况
def get_current_weather(location, unit="摄氏度"):
    # 获取指定地点的天气
    temperature = -1
    if '大连' in location or 'Dalian' in location:
        temperature = 10
    if '上海' in location or 'Shanghai' in location:
        temperature = 36
    if '深圳' in location or 'Shenzhen' in location:
        temperature = 37
    weather_info = {
        "location": location,
        "temperature": temperature,
        "unit": unit,
        "forecast": ["晴天", "微风"],
    }
    return json.dumps(weather_info)

# 封装模型响应函数
def get_response(messages):
    try:
        response = dashscope.Generation.call(
            model='qwen-max',
            messages=messages,
            functions=functions,
            result_format='message'
        )
        return response
    except Exception as e:
        print(f"API调用出错: {str(e)}")
        return None

# 使用function call进行QA
def run_conversation():
    query = "大连的天气怎样"
    messages=[{"role": "user", "content": query}]
    
    # 得到第一次响应
    response = get_response(messages)
    if not response or not response.output:
        print("获取响应失败")
        return None
        
    print('response=', response)
    
    message = response.output.choices[0].message
    messages.append(message)
    print('message=', message)
    
    # Step 2, 判断用户是否要call function
    if hasattr(message, 'function_call') and message.function_call:
        function_call = message.function_call
        tool_name = function_call['name']
        # Step 3, 执行function call
        arguments = json.loads(function_call['arguments'])
        print('arguments=', arguments)
        tool_response = get_current_weather(
            location=arguments.get('location'),
            unit=arguments.get('unit'),
        )
        tool_info = {"role": "function", "name": tool_name, "content": tool_response}
        print('tool_info=', tool_info)
        messages.append(tool_info)
        print('messages=', messages)
        
        #Step 4, 得到第二次响应
        response = get_response(messages)
        if not response or not response.output:
            print("获取第二次响应失败")
            return None
            
        print('response=', response)
        message = response.output.choices[0].message
        return message
    return message

# 这个地方的description，我先用的英文，你可以动手改成中文试试
functions = [
    {
      'name': 'get_current_weather',
      'description': 'Get the current weather in a given location.',
      'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': 'The city and state, e.g. San Francisco, CA'
                },
                'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}
            },
        'required': ['location']
      }
    }
]

if __name__ == "__main__":
    result = run_conversation()
    if result:
        print("最终结果:", result)
    else:
        print("对话执行失败")

response= {"status_code": 200, "request_id": "6560bbda-2a1d-4c71-8758-ca727a029d56", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "function_call", "message": {"role": "assistant", "content": "", "function_call": {"arguments": "{\"location\": \"大连, 辽宁\", \"unit\": \"celsius\"}", "name": "get_current_weather"}}, "index": 0}]}, "usage": {"input_tokens": 275, "output_tokens": 29, "prompt_tokens_details": {"cached_tokens": 0}, "total_tokens": 304}}
message= {"role": "assistant", "content": "", "function_call": {"arguments": "{\"location\": \"大连, 辽宁\", \"unit\": \"celsius\"}", "name": "get_current_weather"}}
arguments= {'location': '大连, 辽宁', 'unit': 'celsius'}
tool_info= {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "\\u5927\\u8fde, \\u8fbd\\u5b81", "temperature": 10, "unit": "celsius", "forecast": ["\\u6674\\u5929", "\\u5fae\\u98ce"]}'}
messages= [{'role': 'user', 'content': '大连的天气怎样'}, Message({'role

## 天气 Function 调用工作流（Qwen + Function Calling）
下面的流程图展示了本 demo 从初始化到得到最终回答的完整路径：

.
```
[Start]
   |
   v
[初始化与鉴权] 导入库 + 设置 dashscope.api_key
   |
   v
[定义本地工具] get_current_weather(location, unit) -> 返回 JSON 字符串
   |
   v
[声明 functions 列表] 以 JSON Schema 描述工具的 name/parameters/required
   |
   v
[构造首问] messages = [{role: 'user', content: '大连的天气怎样'}]
   |
   v
[第一次调用模型] response = get_response(messages)
   |
   v
[取助手消息] message = response.output.choices[0].message
   |
   v
[追加历史] messages.append(message)  # 保持上下文
   |
   v
{ 是否包含 function_call ? } ---- 否 ----> [返回该 message 作为最终结果] --> [End]
              |
             是
              |
              v
[解析调用意图] name = function_call.name; args = JSON.parse(function_call.arguments)
              |
              v
[执行本地工具] tool_response = get_current_weather(args.location, args.unit)
              |
              v
[回填工具结果] messages.append({role:'function', name, content: tool_response})
              |
              v
[第二次调用模型] response = get_response(messages)
              |
              v
[取最终回答] message = response.output.choices[0].message
              |
              v
[End] 打印/返回最终自然语言回答
```
.
要点与数据结构：
- messages：对话历史数组，元素形如 `{role, content}`；函数结果用 `{role:'function', name, content}`。
- function_call：模型在需要外部数据时给出的“调用请求”（包含 `name` 与 JSON 字符串 `arguments`）。
- get_response：统一封装 `dashscope.Generation.call(model='qwen-max', messages=messages, functions=functions, result_format='message')`。
- 容错建议：
  - 访问 `choices[0]` 前检查 `response`/`output`/`choices` 是否存在。
  - `function_call` 可能不存在；解析 `arguments` 用 try/except 包裹。
  - 控制 messages 长度，避免上下文无限增长。